In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import MNIST, ImageFolder

from tqdm.notebook import tqdm

In [2]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
PATH_TO_SAVE_MNIST = '../dataset/'
PATH_IMAGE_TRAIN = '../images/train'
PATH_IMAGE_TEST = '../images/test'

In [4]:
batch_size_train = 64
batch_size_val = 1024

In [5]:
transform_MNIST = T.Compose([
                    T.ToTensor(),
                    T.Normalize((0.1307,), (0.3081,))
                ])

transform = T.Compose([
                T.ToTensor(),
                T.Grayscale()
            ])

train_dataset = MNIST(PATH_TO_SAVE_MNIST, train=True, download=True, transform=transform_MNIST)
test_dataset = MNIST(PATH_TO_SAVE_MNIST, train=False, download=True, transform=transform_MNIST)

train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size_val, shuffle=True)

In [6]:
dataset = ImageFolder(root=PATH_IMAGE_TRAIN, transform=transform)
test_set = ImageFolder(root=PATH_IMAGE_TEST, transform=transform)

# Split
train_set, val_set = torch.utils.data.random_split(dataset, [35000, 5000])

# Create dataloader
train_dl = DataLoader(dataset=train_set, shuffle=True, batch_size=batch_size_train)
val_dl = DataLoader(dataset=val_set, shuffle=True, batch_size=batch_size_val)

test_dl = DataLoader(dataset=test_set, shuffle=False, batch_size=10000)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #input channel 1, output channel 10
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1)
        #input channel 10, output channel 20
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        #dropout layer
        self.conv2_drop = nn.Dropout2d()
        #fully connected layer
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        counter += 1
        tk0.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
from torch.optim import Adam

learning_rate = 0.001
model = CNN().to(DEVICE)
optimizer = Adam(model.parameters(), lr=learning_rate)

num_epoch = 10
for epoch in range(1, num_epoch + 1):
        train(model, DEVICE, train_loader, optimizer, epoch)
        test(model, DEVICE, test_loader)


Test set: Average loss: 0.1613, Accuracy: 9535/10000 (95%)



Test set: Average loss: 0.1279, Accuracy: 9599/10000 (96%)



Test set: Average loss: 0.1154, Accuracy: 9676/10000 (97%)



Test set: Average loss: 0.1007, Accuracy: 9710/10000 (97%)



Test set: Average loss: 0.0919, Accuracy: 9749/10000 (97%)



Test set: Average loss: 0.0865, Accuracy: 9749/10000 (97%)



Test set: Average loss: 0.0760, Accuracy: 9764/10000 (98%)



Test set: Average loss: 0.0795, Accuracy: 9745/10000 (97%)



Test set: Average loss: 0.0732, Accuracy: 9782/10000 (98%)



Test set: Average loss: 0.0720, Accuracy: 9783/10000 (98%)
